<a href="https://colab.research.google.com/github/gowun/BladderCancer_AMC/blob/master/Notebooks/Data_Stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
from google.colab import drive
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', -1)

auth.authenticate_user()
drive.mount('/content/gdrive')
home_path = '/content/gdrive/My Drive/BladderCancer_AMC/'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/gowun/BladderCancer_AMC.git

Cloning into 'BladderCancer_AMC'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 108 (delta 36), reused 64 (delta 17), pack-reused 0
Receiving objects: 100% (108/108), 3.96 MiB | 5.01 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [ ]:
from BladderCancer_AMC.ModelingTools import utils as ut
from BladderCancer_AMC.ModelingTools import preprocessing as pp
from BladderCancer_AMC.ModelingTools import figure as fg

In [ ]:
ds_names = ['mda_mvac', 'mda_ddmvac', 'meta_datasets', 'amc']
datasets4 = list(map(lambda x: ut.load_data(f'{home_path}{x}.pkl', 'pickle'), ds_names))

1. 모든 데이터셋에 존재하는 컬럼 (intersect_cols) 및 각 데이터 내 intersect_cols 비율 

In [ ]:
#### 전체 데이터셋 모두에서 등장하는 컬럼셋
datasets4_cols = list(map(lambda x: set(x.columns), datasets4))
intersect_cols = datasets4_cols[0]
for cols in datasets4_cols[1:]:
  intersect_cols = sorted(set(intersect_cols).intersection(cols))
len(intersect_cols)

17000

In [ ]:
intersect_cols.index('response')

16999

In [ ]:
## 데이터셋 별 intersect_cols 의 비율
for i, cols in enumerate(datasets4_cols):
  r = round(len(intersect_cols) / len(cols) * 100.0, 2)
  print(f'{ds_names[i]}: {r}% from {len(cols)} Gene Symbols')

mda_mvac: 81.98% from 20737 Gene Symbols
mda_ddmvac: 81.7% from 20808 Gene Symbols
meta_datasets: 36.94% from 46023 Gene Symbols
amc: 69.49% from 24463 Gene Symbols


2. 주어진 각 Classifiers 중 intersect_cols에 포함된 비율

In [ ]:
## Classifier.csv 로드 및 정제
clfs = ut.load_data(home_path + 'Classifiers.csv', 'csv').T
clfs

In [ ]:
added = np.array([['ADGRE5', 'AFAP1', 'ATAD3B', 'BGN', 'C11orf95', 'CADM4', 'CARD16', 'CD53', 'CDK6', 'DPH2', 
                   'INHBA', 'ITGAX', 'KCTD1', 'KCTD14', 'KIAA1671', 'MAP7D3', 'MYLK', 'NDUFC2-KCTD14', 'NEK6', 'NT5E', 
                   'OLFM4', 'P3H1', 'PCDHB9', 'PLEK', 'POSTN', 'PPIL2', 'PPP1R35', 'PTPRC', 'RFX7', 'SHANK3', 
                   'SIX5', 'TCAF2', 'TFAM', 'TMEM132A', 'UNC119B', 'YBX3P1'],
                  ['ADGRE5', 'AFAP1', 'ATAD3B', 'BCR', 'BGN', 'BHLHE40', 'C11orf95', 'CADM4', 'CARD16', 'CD52', 
                   'CD53', 'CDH11', 'CDK6', 'COX17', 'DPH2', 'FKBP2', 'FYB', 'GADD45B', 'HCFC1R1', 'INHBA', 
                   'ITGAX', 'KCTD1', 'KCTD14', 'KIAA1671', 'LOC100130744', 'LOC284454', 'LOC648987', 'MAP7D3', 'MYLK' 'NDUFC2-KCTD14', 
                   'NEK6', 'NT5E', 'NUDT12', 'OLFM4', 'P3H1', 'PCDHB9', 'PLEK', 'POSTN', 'POU2F2', 'PPIL2', 
                   'PPP1R35', 'PTPRC', 'RFX7', 'SEPT6', 'SHANK3', 'SIX5', 'SLC15A3', 'TCAF2', 'TFAM', 'THBS2', 
                   'TMEM132A', 'TNFAIP8', 'UBE2T', 'UNC119B', 'YBX3P1']])

In [ ]:
name_list = list(clfs.index)[:-1] + ['GESA_YES_Genes', 'Up_Regulated']
clfs_dict = dict()
for i, k_name in enumerate(name_list):
  if i < len(name_list) - 2:
    tmp = clfs.values[i][np.array(clfs.iloc[i].isna() == False)]
  else:
    ii = i - len(clfs) + 1
    tmp = added[ii]
  tmp1 = list(filter(lambda x: x in intersect_cols, tmp))
  clfs_dict[k_name] = tmp1

In [ ]:
ut.save_data(clfs_dict, home_path + 'intersect_classifiers.pkl', 'pickle')

In [ ]:
clfs_df = pd.DataFrame(clfs_dict.values(), index=list(clfs_dict.keys())).T
ut.save_data(clfs_df, home_path + 'intersect_classifiers.csv', 'csv')

3. 각 데이터셋의 기초 통계

In [ ]:
%%time
mean_datasets4 = list(map(lambda x: pp.handle_repeated_columns(x[intersect_cols])[1], datasets4))

CPU times: user 44 s, sys: 610 ms, total: 44.6 s
Wall time: 44 s


In [ ]:
ut.save_data(mean_datasets4, f'{home_path}mean_datasets4.pkl', 'pickle')

In [ ]:
%%time
for i, nm in enumerate(ds_names):
  tmp = mean_datasets4[i].describe()
  ut.save_data(tmp.reset_index(), f'{home_path}{nm}.csv', 'csv')

CPU times: user 2min 3s, sys: 560 ms, total: 2min 3s
Wall time: 2min 6s


4. Optimal Normalization Algorithm 도출

In [ ]:
#methods = ['log2', 'log2_minmax', 'standard', 'minmax', 'max_abs', 'robust', 'power', 'quantile', 'rankgauss']
methods = ['standard', 'power', 'rankgauss']

In [ ]:
%%time
scaled_datasets4 = []
for i, d in enumerate(mean_datasets4):
  print(i)
  tmp = pp.scale_with_scalers(d.drop(['response'], 1), methods)
  scaled_datasets4.append(pp.scale_with_scalers(d, methods))

In [ ]:
from itertools import combinations, chain
orders = list(range(len(mean_datasets4)))
orders = list(combinations(orders, 2))
all_genes_in_clas = list(set(chain(*list(clfs_dict.values()))))
orders, all_genes_in_clas

([(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)],
 ['TP53',
  'SYP',
  'SPRR2D',
  'KCNH2',
  'SPINK1',
  'TNIP3',
  'LTK',
  'CD3D',
  'FMO9P',
  'MSI1',
  'SORBS1',
  'PDGFRA',
  'CYP2J2',
  'TGFBI',
  'HTT',
  'ADA',
  'CDH12',
  'SERPINB13',
  'ART5',
  'NCOR1',
  'ZCCHC12',
  'PDZRN4',
  'ZMAT3',
  'CCNB2',
  'ERN2',
  'GPX3',
  'CCL5',
  'MYCN',
  'CSTA',
  'RHCG',
  'B3GAT1',
  'TWIST1',
  'ABI3BP',
  'RPL9',
  'POU2F2',
  'S100A8',
  'DACT3',
  'KRT20',
  'MCM7',
  'TFDP1',
  'ANXA2',
  'NRCAM',
  'SMC2',
  'CD3E',
  'FEN1',
  'DES',
  'AOX1',
  'BTBD16',
  'SLC30A2',
  'PM20D2',
  'DPF1',
  'NFKB1',
  'KRT81',
  'GABBR2',
  'PIAS4',
  'SDK2',
  'HDC',
  'MS4A1',
  'ADAM33',
  'CD24',
  'ENHO',
  'TMEM63A',
  'ACTG2',
  'CD247',
  'PRNP',
  'SGCD',
  'CASQ2',
  'MUC2',
  'CHEK1',
  'LMO3',
  'RB1',
  'ZEB2',
  'IGFBP5',
  'RPAP1',
  'UHRF1',
  'FABP4',
  'HMGA2',
  'SRF',
  'UNC119B',
  'PCP4',
  'NOD2',
  'DPT',
  'ENO2',
  'SFN',
  'DCAKD',
  'SNAP91',
  'VSIG8',
  'RPL26L1'

In [ ]:
%%time
from scipy.stats import normaltest, ttest_ind_from_stats
def compare_two_samples_and_draw_feasible_columns(sample1, sample2, min_pvalue=0.01):
    col1 = set(sample1.columns)
    col2 = set(sample2.columns)
    interset_cols = sorted(col1.intersection(col2))

    passed_cols = []
    for c in interset_cols:
        # For a different distribution, we can reject the null hypothesis since the pvalue is below 1%
        # This is a two-sided test for the null hypothesis that two independent samples have identical average (expected) values
        s1, s2 = sample1[c], sample2[c]
        t, pvalue = ttest_ind_from_stats(np.mean(s1), np.std(s1), len(s1), np.mean(s2), np.std(s2), len(s2))
        if pvalue >= min_pvalue:
            passed_cols.append(c)
    
    return passed_cols

all_pass_genes_method = {'standard': [], 'power': [], 'rankgauss': []}
for m in methods:
  tmp_dict = dict()
  for o1, o2 in orders:
    cols = compare_two_samples_and_draw_feasible_columns(scaled_datasets4[o1][m][all_genes_in_clas], scaled_datasets4[o2][m][all_genes_in_clas])
    for c in cols:
      if c in tmp_dict.keys():
        tmp_dict[c] += 1
      else:
        tmp_dict[c] = 1
  print(tmp_dict)
  for c, s in tmp_dict.items():
    if s == len(orders):
      all_pass_genes_method[m].append(c)
for m in methods:
  print(m, len(all_pass_genes_method[m]), len(all_genes_in_clas))

{'ABI3BP': 6, 'ACTA2': 6, 'ACTC1': 6, 'ACTG2': 6, 'ADA': 6, 'ADAM33': 6, 'ADAMTS18': 6, 'ADCY5': 6, 'ADD2': 6, 'ADH1B': 6, 'ADH6': 6, 'AFAP1': 6, 'AGAP11': 6, 'AHNAK2': 6, 'AHR': 6, 'ANLN': 6, 'ANXA2': 6, 'ANXA8': 6, 'AOX1': 6, 'APLP1': 6, 'AREG': 6, 'ART5': 6, 'ASPM': 6, 'ATAD2': 6, 'ATAD3B': 6, 'ATOH8': 6, 'ATP1A2': 6, 'AURKA': 6, 'AURKB': 6, 'B3GAT1': 6, 'BAMBI': 6, 'BARX2': 6, 'BCL2A1': 6, 'BCR': 6, 'BEX1': 6, 'BGN': 6, 'BHMT': 6, 'BLK': 6, 'BRSK1': 6, 'BRSK2': 6, 'BTBD16': 6, 'BUB1': 6, 'C19orf45': 6, 'C1orf61': 6, 'C7': 6, 'CA9': 6, 'CADM4': 6, 'CARD16': 6, 'CASQ2': 6, 'CCDC64': 6, 'CCL19': 6, 'CCL2': 6, 'CCL20': 6, 'CCL5': 6, 'CCL8': 6, 'CCNA2': 6, 'CCNB1': 6, 'CCNB2': 6, 'CCND2': 6, 'CCNE1': 6, 'CCNE2': 6, 'CCNG1': 6, 'CCR7': 6, 'CD109': 6, 'CD24': 6, 'CD247': 6, 'CD274': 6, 'CD38': 6, 'CD3D': 6, 'CD3E': 6, 'CD3G': 6, 'CD44': 6, 'CD52': 6, 'CD53': 6, 'CD82': 6, 'CDA': 6, 'CDH11': 6, 'CDH12': 6, 'CDH3': 6, 'CDK6': 6, 'CDKN2A': 6, 'CDKN3': 6, 'CDSN': 6, 'CHD2': 6, 'CHEK1': 6, 'CH

In [ ]:
%%time
remain_mth_datasets = dict()
for m in methods:
  print(m)
  tmp = []
  for i, ll in enumerate(scaled_datasets4):
    tmp1 = ll[m][all_pass_genes_method[m]]
    tmp1['response'] = list(mean_datasets4[i]['response'])
    tmp.append(tmp1)
  remain_mth_datasets[m] = tmp

standard
power
rankgauss
CPU times: user 32.6 ms, sys: 0 ns, total: 32.6 ms
Wall time: 32 ms


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
ut.save_data(remain_mth_datasets, f'{home_path}scaled_datasets_3mths.pkl', 'pickle')